# Upload a Bode measurement to Flojoy Cloud (with use of an oscilloscope)

Download the Flojoy Cloud Python SDK:
`pip install flojoy_cloud`

Upload a new measurement:
https://rest.flojoy.ai/measurement/create-measurement

Create a free Flojoy Cloud account: https://cloud.flojoy.ai/signup

Deploy Flojoy Cloud to your own AWS ☁️: https://docs.flojoy.ai/cloud/deploy-to-aws/

In [1]:
from flojoy_cloud import FlojoyCloud
import os
import pyvisa
import numpy as np
import pandas as pd
from datetime import datetime
from time import sleep
import plotly.express as px

2024-01-26 12:32:10.283 - Opening DeviceManager


In [2]:
# Find available VISA devices
rm = pyvisa.ResourceManager('@py')
devices = rm.list_resources()
# print(devices)

In [ ]:
# Connect to the oscilloscope
inst = rm.open_resource(devices[0])
inst.read_termination = "\n"
inst.write_termination = "\n"

In [22]:
# Functions used
def device_power():
    return inst.query("*IDN?")


def v_to_db(volts, ref_voltage):
    """
    Convert from voltage to dB.
    """
    volts = np.array(volts)
    return 20 * np.log10(volts / ref_voltage)


def scope_data():
    """
    Test that varies signal frequency to create a Bode plot.
    """
    inst.write(":MEAS:STAT:RESet")
    inst.write(":MEAS:STAT:ITEM vpp,CHAN1")
    freqs = np.logspace(5, 7, base=10, num=50, endpoint=True)
    volts = []
    df = pd.DataFrame()

    for i in freqs:
        inst.write(f"SOUR1:FREQ {i}")
        inst.write(":MEAS:STAT:RESet")
        sleep(0.5)
        measure = float(inst.query(":MEAS:STAT:ITEM? AVER,VPP,CHAN1"))
        volts.append(measure)


    # Plot
    df["x"] = freqs
    df["y"] = v_to_db(volts, 1.2)
    fig = px.line(
        df,
        x="x",
        y="y",
        log_x=True,
    )
    fig.show()

    return df


def scope_bool(df):
    """
    Test to ensure the signal above the low pass filter is strong, and below
    the signal should be weak.
    """
    return (df["y"].max() > -1) and (df["y"].min() < -35)

In [4]:
# Client with secret from cloud.flojoy.ai (in Settings)
client = FlojoyCloud(workspace_secret=os.getenv("flojoy_cloud_token"))

In [5]:
# Choose workspace_id
workspace_id = "workspace_khsvq1jvk76uq5ltf6atdsvi"

In [6]:
# Create a device (or system) model
device_model = client.create_device_model(
    name="DS1234", workspace_id=workspace_id
)

In [7]:
# Create a project with the model
project = client.create_project(
    datetime.now().strftime("%Y%m%d"), model_id=device_model.id, workspace_id=workspace_id
)

In [8]:
# Create tests
test_bool = client.create_test(
    "DS scope bool", project_id=project.id, measurement_type="boolean"
)
test_df = client.create_test(
    "DS scope df", project_id=project.id, measurement_type="dataframe"
)

In [17]:
# Create devices
device1 = client.create_device(
    workspace_id=workspace_id, name="SN0003", model_id=device_model.id
)
client.add_hardware_to_project(
    project_id=project.id, hardware_id=device1.id
)

device2 = client.create_device(
    workspace_id=workspace_id, name="SN0004", model_id=device_model.id
)
client.add_hardware_to_project(
    project_id=project.id, hardware_id=device2.id
)

In [26]:
# Perform tests
data = scope_data()
passed = scope_bool(data)

In [ ]:
# Upload measurements
client.upload(data=passed, test_id=test_bool.id, hardware_id=device1.id)
client.upload(data=data, test_id=test_df.id, hardware_id=device1.id)

In [28]:
# Perform tests for second device
data = scope_data()
passed = scope_bool(data)

In [ ]:
# Upload measurements
client.upload(data=passed, test_id=test_bool.id, hardware_id=device2.id)
client.upload(data=data, test_id=test_df.id, hardware_id=device2.id)

In [ ]:
# Close the instrument when finished
inst.close()

Now, go to your project at https://cloud.flojoy.ai/workspace/. From here, you can look at all measurements